In [1]:
# encoding: utf-8
import urllib.request
import requests
import pdb
import xmltodict
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

KEY = "1fa61c56392e94a6"
COUNT = 100
# グルメサーチAPI
H_API = "https://webservice.recruit.co.jp/hotpepper/gourmet/v1/" #?key=sample&large_area=Z011
# google検索
G_SEARCH = "https://www.google.co.jp/search?num=5&q="

def hotpepper_search(keywords, start=1):
    """
    keywords : list of str
    d : dict
    """
    keys = {
        "key": KEY,
        "genre": " ".join(keywords),
        "large_area": "Z011", # 東京
        "start": start,
        "count": COUNT
    }
    url = "{}?{}".format(H_API, urllib.parse.urlencode(keys))
    req = urllib.request.Request(url)
    try:
        with urllib.request.urlopen(req) as res:
            body = res.read()
        d = xmltodict.parse(body)
        return d
    except Exception as e:
        print(e)
        pdb.set_trace()

def google_search(keys):
    """
    keys : list or str
    url : 検索結果最上位のurl
    """
    url = G_SEARCH + " ".join(keys)
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")
        for el in soup.select("h3.r a"):
            title = el.get_text()
            print(title)
            url = dict(urllib.parse.parse_qsl(urllib.parse.urlparse(el.get("href")).query))["q"]
            break
        return url
    except Exception as e:
        print(e)
        return ""

def tabelog_search(url):
    if "tabelog" not in url:
        print("not found tabelog page!")
        return None
    try:
        sleep(3)
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")
        score = float(soup.find_all("span", class_="rdheader-rating__score-val-dtl")[0].string)
        print("score : {}".format(score))
        return score
    except Exception as e:
        print(e)
        print("not found tabelog score")
        return None
if __name__=="__main__":
    df = pd.read_csv("data/test.csv", encoding="cp932", header=0).set_index("name")
    shop_ids = set(df["id"])
    keys = ["G005"]
    # keys = ["バル"]
    # keys = ["居酒屋"]
    out = hotpepper_search(keys)
    num = int(out["results"]["results_available"])
    print("{} shops available in hotpepper".format(num))
    shops = []
    scores = []
    ind = 0
    try:
        for ind in range(0,num,COUNT):
            print("{} / {}".format(ind, num))
            out = hotpepper_search(keys, start=ind)
            tmp_shops = []
            scores = []
            for shop in out["results"]["shop"]:
                try:
                    if shop["id"] in shop_ids:
                        print("{} already in data".format(shop["name"]))
                        continue
                    url = google_search(["食べログ", shop["name"]])
                    score = tabelog_search(url)
                    if score is not None:
                        tmp_shops.append(shop)
                        scores.append(score)
                except Exception:
                    pass
            if tmp_shops:
                df2 = pd.DataFrame(tmp_shops)
                df2["score"] = scores
                df2 = df2.set_index("name")
                df = pd.concat((df, df2))
                df.to_csv("data/test.csv", encoding="cp932")
            
    # out2 = google_search(["信濃町", "煉瓦"])
    except KeyboardInterrupt:
        pass
    df2 = pd.DataFrame(shops)
    df2["score"] = scores
    df2 = df2.set_index("name")
    df3 = pd.concat((df, df2))
    df3.to_csv("data/test.csv", encoding="cp932")
    pdb.set_trace()


825 shops available in hotpepper
0 / 825
神田の肉バル ランプキャップ RUMP CAP 池袋西口店 already in data
スコール SQUALL 東新宿店 already in data
MEAT&WINE ワインホールグラマー 池袋 - 池袋/イタリアン [食べログ]
score : 3.12
レ クアトロ スタジオーニ （Le quattro stagioni） - 京成上野 ... - 食べログ
score : 3.33
夜カフェ アロハランド 新宿東口店 - 新宿/居酒屋 [食べログ]
score : 3.05
フォンティーナ チーズ ダイニング （Fontina Cheese Dining ... - 食べログ
score : 3.21
ママ食堂 市ヶ谷店 （ママショクドウ） - 市ヶ谷/居酒屋 [食べログ]
score : 3.07
ステーキハウス ハマ 銀座店 - 新橋/ステーキ [食べログ]
score : 3.19
LIT STEAK HOUSE GINZA - 東銀座/ステーキ [食べログ]
could not convert string to float: '-'
not found tabelog score
焼肉ダイニング bonbori 渋谷道玄坂店 - 渋谷/焼肉 [食べログ]
score : 3.06
個室ダイニング お肉で宴会 新宿西口店 - 新宿/居酒屋 [食べログ]
could not convert string to float: '-'
not found tabelog score
神田の肉バル ランプキャップ RUMP CAP 銀座店 already in data
PIZZA&WINE ワインホールグラマー 新宿 - 新宿三丁目 ... - 食べログ
score : 3.06
神田の肉バル ランプキャップ RUMP CAP 渋谷店 already in data
シュラスコレストラン ALEGRIA shinjuku（アレグリア） - 代々木 - 食べログ
score : 3.29
神田の肉バル ランプキャップ RUMP CAP 田町店 already in data
広島風お好み 鉄板焼き 友 - 新宿三丁目/お好み焼き [食べログ]
score : 3.34


c:\anaconda\lib\site-packages\ipykernel_launcher.py:108: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



100 / 825
NORI （ノリ） - 白金高輪/イタリアン [食べログ]
score : 3.05
銀座ライオン 霞が関コモンゲート店 - 虎ノ門/ビアホール・ビア ... - 食べログ
score : 3.07
神戸ビーフ食品直営店 鉄板焼 銀座888 - 汐留/鉄板焼き [食べログ]
score : 3.04
レッドロブスター 上板橋店 （Red Lobster） - 上板橋/シーフード [食べログ]
score : 3.51
焼肉ほりたん - 銀座/焼肉 [食べログ]
score : 3.52
銀座 ドギーズ already in data
テルポリート from ユキノヤ （teruporito from ユキノヤ） - 板橋 ... - 食べログ
score : 3.05
黒毛和牛専門店ステーキモンスター 日本橋黒桜 - 水天宮前 ... - 食べログ
score : 3.09
アルトマーレ - 池尻大橋/イタリアン [食べログ]
score : 3.03
赤羽 union already in data
オイスターテーブル Oystar Table 浜松町店 already in data
コルザ - 銀座/鉄板焼き [食べログ]
score : 3.58
銀座ライオン 上野西郷会館店 - 京成上野/ビアホール・ビア ... - 食べログ
score : 3.07
ステーキダイニング 大地の恵み 上野御徒町店 already in data
東京のおすすめステーキ [食ベログ] - 食べログ
list index out of range
not found tabelog score
鉄板焼き かいか already in data
エイジング・ビーフ TOKYO 新宿三丁目店 - 食べログ
score : 3.51
すうぷ屋 新宿ミロード店 （スウプヤ） - 新宿/スープ [食べログ]
score : 3.1
鉄板個室ダイニング ぽんぽこ八兵衛 - 門前仲町/お好み焼き [食べログ]
could not convert string to float: '-'
not found tabelog score
ステーキ パンドラ 匠 （タクミ） - 新宿/ステーキ [食べログ]
could not convert string to float:

KeyError: 'name'